## Bell, T1, T2 Parallel Experiment
This is an experiment to Run T1/T2 Experiment device-wide, and also to run parallel Bell state measurements as a simple proxy for two-qubit gate fidelity. Note this measure of Hellinger fidelity from the Bell test is sensitive to state preparation and measurement (SPAM) error as well.

In [ ]:
# %load_ext autoreload
# %autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rustworkx as rx

In [ ]:
import sys # probably a better practice than this but here it is
sys.path.append('../')

In [ ]:
import qiskit_device_benchmarking

In [ ]:
from qiskit_ibm_provider import IBMProvider

provider =IBMProvider()
backend = provider.get_backend('ibm_torino')
backend.status()

In [ ]:
nq = backend.configuration().n_qubits
coupling_map = backend.configuration().coupling_map


In [ ]:
from qiskit_aer import AerSimulator

backend_sim = AerSimulator.from_backend(backend)
backend = backend_sim

In [ ]:
import qiskit_device_benchmarking.utilities.graph_utils  as gu #import paths_flatten,remove_permutations,path_to_edges,build_sys_graph,get_separated_sets
#build a set of gates
G = gu.build_sys_graph(nq, coupling_map)
#get all length 2 paths in the device
paths = rx.all_pairs_all_simple_paths(G,2,2)
#flatten those paths into a list from the rustwork x iterator
paths = gu.paths_flatten(paths)
#remove permutations
paths = gu.remove_permutations(paths)
#convert to the coupling map of the device
paths = gu.path_to_edges(paths,coupling_map)
#make into separate sets
sep_sets = gu.get_separated_sets(G, paths, min_sep=2)

qubits_nns = gu.get_iso_qubit_list(G)

In [ ]:
from qiskit_experiments.library import T2Ramsey,Tphi,T1,StandardRB

# Time intervals to wait before measurement for t1 and t2
delays_t1 = np.arange(1e-6, 300e-6, 15e-6)
delays_t1 = [0] + list(np.logspace(np.log10(10e-9),np.log10(200e-6),41))

delays_t2 = [0] + np.arange(1e-6, 300e-6, 15e-6).tolist()
num_periods = 5
max_T = delays_t2[-1]
osc_freq = num_periods/(max_T)

In [ ]:
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment

exp_batches = []
for qubits in qubits_nns:
    coh_exps = ParallelExperiment([
        Tphi((int(qubit),), delays_t1=delays_t1,delays_t2=delays_t2,t2type='ramsey',osc_freq=osc_freq,backend=backend)
        for qubit in np.array(qubits).flatten()
    ], flatten_results=True)
    exp_batches.append(coh_exps)


In [ ]:
from qiskit_device_benchmarking.bench_code.bell.bell_experiment import BellExperiment

bell_exp = BellExperiment(sep_sets,backend=backend)

In [ ]:
exp_batches.append(bell_exp)

In [ ]:
batch_exp = BatchExperiment(exp_batches,backend=backend,flatten_results=True)

In [ ]:
batch_exp_data = batch_exp.run()

In [ ]:
batch_exp_data.status()

In [ ]:
i=1
print(batch_exp_data.figure(i).metadata)
batch_exp_data.figure(i).figure

In [1]:
from IPython.display import HTML, display
import datetime
def qiskit_copyright(line="", cell=None):
    """IBM copyright"""
    now = datetime.datetime.now()

    html = "<div style='width: 100%; background-color:#d5d9e0;"
    html += "padding-left: 10px; padding-bottom: 10px; padding-right: 10px; padding-top: 5px'>"
    html += "<p>&copy; Copyright IBM 2017, %s.</p>" % now.year
    html += "<p>This code is licensed under the Apache License, Version 2.0. You may<br>"
    html += "obtain a copy of this license in the LICENSE.txt file in the root directory<br> "
    html += "of this source tree or at http://www.apache.org/licenses/LICENSE-2.0."

    html += "<p>Any modifications or derivative works of this code must retain this<br>"
    html += "copyright notice, and modified files need to carry a notice indicating<br>"
    html += "that they have been altered from the originals.</p>"
    html += "</div>"
    return display(HTML(html))

qiskit_copyright()